# Local Sidereal Time and j0

#This code calculates Local Sidereal Time and j0 value with respect to UTAA observation location. Results are given in the end.
#In writing this code, for some parts M-files provided by Howard Curtis are directly used (LST.m). These uses are cited at the related points. And some of them mentioned in Turkish.

# In this code we need also TLE data.
#1 25544U 98067A 18360.90043441 .00000834 00000-0 19881-4 0 9999
#2 25544 51.6409 138.3193 0004560 186.2932 317.3191 15.54096287148488
#Taking this TLE data or providing TLE's from space-track.org, you can use below code for Python to find Classical Orbital #Elements. You need to copy TLE data and paste to a text file as a name 'tles'. And they both has to be in same folder with the #code.

In [ ]:
def TLE_to_COE_and_t0():

    fileID = open('tles.txt', 'r')
    fileID.seek(18)
    epoch = float(fileID.read(5))
    fileID.seek(40)
    n0dot = 2.0*(float(fileID.read(3))/1000000)
    fileID.seek(45)
    n0doubledot1 = 6.0*(float(fileID.read(1)))
    fileID.seek(51)
    n0doubledot2 = 10**float(fileID.read(1))
    n0doubledot = n0doubledot1*n0doubledot2
    fileID.seek(54)
    Bstar1 = float(fileID.read(5))
    fileID.seek(60)
    Bstar2 = 10**(-float(fileID.read(1)))
    Bstar = Bstar1*Bstar2
    fileID.seek(80)
    i = float(fileID.read(7))*math.pi/180.0
    fileID.seek(88)
    Omega = (float(fileID.read(8)))*math.pi/180.0
    fileID.seek(100)
    e = (float(fileID.read(4)))/(10**7)
    fileID.seek(105)
    omega = (float(fileID.read(8)))*math.pi/180.0
    fileID.seek(114)
    Me = (float(fileID.read(8)))*math.pi/180.0
    fileID.seek(123)
    # Ortalama hareket, rad/s biriminde
    n = (float(fileID.read(7)))*2.0*math.pi/(24.0*3600.0)

    #Klasik yörünge elemanlarının bulunuşu, bunlar; h, i, Omega, e, omega ,theta
    #TLE data çıktıları. i, Omega, e, omega TLE dosyasından okunabilir. h ve theta,
    #n ve Me'ye göre bulunur.

    #Kütleçekimsel parametre mu, km^3/s^2:
    mu = 398600

    #h'ın n'den elde edilişi:
    h = (mu**2/n)**(1/3)*math.sqrt(1-e**2)
    

    #Kepler denklemini çözerek theta'nın Me'den elde edilişi:
  
    def f(E):
            KeplerEqn = E - e*math.sin(E) - Me
            return KeplerEqn
    #Kepler denkleminin çözümü, başlangıç tahmini Pi.
            E = fsolve(f,[math.pi])
            
            if math.tan(E/2.0)<0:
                theta = 2.0*math.atan(math.sqrt((1+e)/(1-e))*math.tan(E/2.0)) + 2.0*math.pi;
            else:
                theta = 2.0*math.atan(math.sqrt((1+e)/(1-e))*math.tan(E/2.0));
                #Klasik yörünge elemanları(kye). Ortalama hareketin birimi,
                #rad/s ve bütün açılar radyan cinsinden.
                kye = numpy.array([h, i, Omega, e, omega, theta])
                print (kye)
    
    #Çağ, Evrensel Zamanı tanımlar (Greenwich'teki güneş zamanı). Aşağıda, 
    #çağ yıla çevrilmiştir, yılın günü gün kesri ile
    
    year = 2000 + (epoch - numpy.mod(epoch,1000))/1000
    
    day = numpy.mod(epoch,1000)

    #Bu fonksiyon yıldız zamanını hesaplar.
    
    #lst  -yıldız zamanı (derece)
    #y    -yıl 
    #m    -ay
    #d    -day
    #ut   -Evrensel Zaman
    #EL   -Doğu boylamı
    #j0   -Jülyen günü sayısı, 0'ıncı saat UT
    #j    -J2000'den beri olan yüzyıl sayısı
    #g0   -Greenwich yıldız zamanı (derece), 0'ıncı saat UT
    #gst  -Greenwich yıldız zamanı (derece), belirlenmiş UT
    
    #Denklem 5.48:
    #Bu fonksiyon Jülyen günü sayısını 0 UT ve 1900 ile 2100 arasındaki herhangi bir yıl için 
    #Denklem 5.48'i kullanarak hesaplar.
    
    #j0     -Jülyen günü, 0 UT (Evrensel Zaman/Universal Time)
    #year   -aralık: 1901 - 2099
    #month  -aralık: 1 - 12
    #day    -aralık: 1 - 31
    month=1

    j0 = 367*year - round(7*(year + round((month + 9)/12))/4) + round(275*month/9) + day + 1721013.5
    print('j0 = ',j0)
    
    
    #Denklem 5.49:
    j = (j0 - 2451545)/36525
    
    #Denklem 5.50:
    g0 = 100.4606184 + 36000.77004*j + 0.000387933*j**2 - 2.583e-8*j**3
    
    #g0'nun 0 ile 360 arasında olması için düşürülmesi
    #g0 = zeroTo360(g0)
    global date
    datenumnew= date.toordinal(date(int(year),int(1),int(30)))+1/8+366
    daysnew = datenumnew % 1
    hoursnew = daysnew % 1 * 24
    minutesnew = hoursnew % 1 * 60
    secondsnew = minutesnew % 1 * 60    
    UTnew = hoursnew + minutesnew/60 + secondsnew/3600
    datenew=datetime.fromordinal(int(datenumnew))+ timedelta(days=int(daysnew)) +timedelta(hours=int(hoursnew)) + timedelta(minutes=int(minutesnew))+ timedelta(seconds=round(secondsnew)) - timedelta(days=366)
    #Denklem 5.51:
    gst = g0 + 360.98564724*UTnew/24;
    #Dünya'nın yarıçapı RE (km) ve Dünya'nın basıklığı (birimsiz)
    RE = 6378
    f = 0.00335

    #Gözlem noktası:
    EL = 32.6890
    Lat = 39.9455
    H = 0.810

    #Denklem 5.52:
    lst = gst + EL;

    #lst'nin 0 ile 360 derece arasına düşürülmesi:
    lst = lst - 360*numpy.fix(lst/360);
    print ('lst = ',lst)
TLE_to_COE_and_t0()